**Bollinger Bands Trading Strategy Optimization, with specific 3x leveraged Bull/ Bear ETFs.** </br>

Examples - Inverse ETF pairs (options available) - TQQQ, SQQQ, SPXL, SPU, SPY, SH, QQQ, PSQ, IWM, RWM, SOXL, SOXS, FAS, FAZ, LABD, LABU </br>
https://www.direxion.com/etfs?filter=bullbear3x

In [57]:
#!pip install yfinance --upgrade
#!pip install talib-binary
#!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
#!tar -xzvf ta-lib-0.4.0-src.tar.gz
#%cd ta-lib
#!./configure --prefix=/usr
#!make
#!make install
#!pip install Ta-Lib

In [58]:
import os
import sys
import copy
import datetime
import time
import itertools

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import yfinance as yf
import talib as ta
from talib import MA_Type

**Defining Data Folder Structures**

In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
#data folder structures
data_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data'
backtest_output_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/backtest_output'

In [61]:
# pair inverse trade stocks
ticker_list = ['TQQQ', 'SQQQ', 'SPXL', 'SPXS', 'TZA', 'TNA', 'SOXL', 'SOXS', 'FAS', 'FAZ', 'LABD', 'LABU', 'EDZ', 'EDC']

# define date range
start_date = '2021-03-20'
end_date = '2022-12-31'
interval = '1d'

# initial capital in USD
initial_capital = 7000

# position sizing
pos_size = round(initial_capital/len(ticker_list),2)

**2) Getting OHLC Data from Yahoo Finance** </b>


*   Get stock data based on ticker list from Yahoo Finance
*   Create each stock data as different dataframe
*   Add Technical Analysis indicators from TA-Lib
*   Pointing working directory to data folder, export dach dataframe as independent csv file 





In [62]:
# changing working directory to data folder's path
data_folder = '/content/drive/MyDrive/Colab Notebooks/Algorithmic trading/data'
os.chdir(data_folder)

# getting OHLC data from yfinance package, if auto_adjust=True, OHLC data will not have adj close column, use progress=False to get rid of comments
# Other indicators: 14 day RSI
for ticker in ticker_list:
  try:
    globals()[ticker] = pd.DataFrame
    globals()[ticker] = yf.download(ticker, start=start_date, end=end_date, interval=interval, auto_adjust=True, back_adjust=True, progress=False)
    globals()[ticker]['200EMA'] = ta.EMA(globals()[ticker]['Close'], timeperiod=200) 
    globals()[ticker]['ATR'] = ta.ATR(globals()[ticker]['High'], globals()[ticker]['Low'], globals()[ticker]['Close'], timeperiod=14)
    globals()[ticker]['RSI_14'] = ta.RSI(globals()[ticker]['Close'], timeperiod=14)
    globals()[ticker]['MACD'], globals()[ticker]['MACD_signal'], globals()[ticker]['MACD_hist'] = ta.MACD(globals()[ticker]['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    globals()[ticker]['ADX'] = ta.ADX(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14)
    globals()[ticker]['MFI'] = ta.MFI(globals()[ticker]['High'], globals()[ticker]['Low'], globals()[ticker]['Close'], globals()[ticker]['Volume'], 14)
    globals()[ticker]['WILL%R'] = ta.WILLR(globals()[ticker]['High'], globals()[ticker]['Low'], globals()[ticker]['Close'], timeperiod=14)

    # Bollinger Bands
    globals()[ticker]['BBand_U'], globals()[ticker]['BBand_M'], globals()[ticker]['BBand_L'] = ta.BBANDS(globals()[ticker]['Close']) # if add exponential smoothing, matype=MA_Type.T3

    # add/drop columns and clean up
    #globals()[ticker] = globals()[ticker].drop(['Volume', 'MACD_hist'], axis=1)
    globals()[ticker] = globals()[ticker].round(4).dropna()
    globals()[ticker] = globals()[ticker].reset_index()
    globals()[ticker]['Position'] = 0
    globals()[ticker][['Lot Size', 'Equity Value', 'MDD_dollar', 'PNL', 'Holding Period', 'Win Count']] = np.NAN
    
    #writing each ticker dataframe into separate .csv files, save to google drive
    #globals()[ticker].to_csv(ticker+'.csv', index=False, header=True)
  except:
    print(globals()[ticker])

**Other Optional Indicators:** </b>

**MACD_hist** - globals()[ticker]['MACD'], globals()[ticker]['MACD_signal'], globals()[ticker]['MACD_hist'] = ta.MACD(globals()[ticker]['Close'], fastperiod=12, slowperiod=26, signalperiod=9) </b>

**ADX** - globals()[ticker]['ADX'] = ta.ADX(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14) </b>


**ADXR** - globals()[ticker]['ADXR'] = ta.ADXR(globals()[ticker]['High'], globals()[ticker]['Open'], globals()[ticker]['Close'], 14) </b>

**50 Day Exponential Moving Average** - globals()[ticker]['50EMA'] = ta.EMA(globals()[ticker]['Close'], timeperiod=50) </b>

**Bollinger Bands** - globals()[ticker]['BBand_U'], globals()[ticker]['BBand_M'], globals()[ticker]['BBand_L'] = ta.BBANDS(globals()[ticker]['Close']) # if add exponential smoothing, matype=MA_Type.T3

**Stochastic Indicators** - globals()[ticker]['Stoch_slowk'], globals()[ticker]['Stoch_slowd'] = ta.STOCH(globals()[ticker]['High'], globals()[ticker]['Low'], globals()[ticker]['Close'],                                                              fastk_period=21, slowk_period=5, slowk_matype=0, slowd_period=5, slowd_matype=0)    

**3) Building the DataFrame List, Perform Backtesting**

In [63]:
# list of dataframes to be iterated after yfinance data pull           
df_list = [TQQQ, SQQQ, SPXL, SPXS, TZA, TNA, SOXL, SOXS, FAS, FAZ, LABD, LABU, EDZ, EDC]

In [64]:
# function to extract dataframe name
def get_df_name(df):
   name =[x for x in globals() if globals()[x] is df][0]
   return name

In [65]:
# function to print function name as a string
def get_func_name(func_name):
    func_name = sys._getframe().f_code.co_name
    return func_name

**MACD Bollinger Bands Strategy**

In [66]:
# defining the global list for storing trade list output
global bb_macd_tradelist
bb_macd_tradelist = []

global bbadxrsi_trade_list
bbadxrsi_trade_list = []

pos_size = round(initial_capital/len(ticker_list),2)

In [67]:
# test condition
def backtest_bbmacd(df):
  df_name = get_df_name(df)

  pos_opened = False
  open_price  = 0
  close_price = 0
  lot_size = 0

  initial_equity_value = 0
  hold_counter = 0
  win_counter = 0

  pnl = 0
  pnl_list = []
    
  for i in range(len(df)):
    now_date        = df.loc[i, 'Date']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_200EMA      = df.loc[i, '200EMA']
    now_rsi_14      = df.loc[i, 'RSI_14']
    now_MACD        = df.loc[i, 'MACD']
    now_MACD_signal = df.loc[i, 'MACD_signal']
    now_MACD_hist   = df.loc[i, 'MACD_hist']
    now_bband_u     = df.loc[i, 'BBand_U']
    now_bband_m     = df.loc[i, 'BBand_M']
    now_bband_l     = df.loc[i, 'BBand_L']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']

    # opening a position
    if (pos_opened == False and (now_open < now_bband_l and now_MACD > now_MACD_signal and now_MACD_hist > 0)):
      pos_opened = True
      open_price = now_open
      lot_size = pos_size//open_price
      buy_commission = (0.0049 + 0.005) * lot_size
      initial_equity_value = round(((lot_size * open_price) - buy_commission),2)
      hold_counter = hold_counter + 1
      df.at[i, 'Position'] = 1
      df.at[i, 'Lot Size'] = lot_size
      df.at[i, 'Equity Value'] = initial_equity_value
      bb_macd_tradelist.append([now_date, df_name, open_price, pos_opened, lot_size, initial_equity_value, 0, 0])

    # closing a position - by MACD signal, or last day of trading or hit stop loss
    elif (pos_opened == True and ((now_MACD < now_MACD_signal) or \
                                  (lot_size * now_open > initial_equity_value * 1.2) or (lot_size * now_open < initial_equity_value * 0.95))):
      pos_opened = False
      close_price = now_open
      sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      df.at[i, 'Position'] = 2
      df.at[i, 'Lot Size'] = lot_size
      df.at[i, 'Equity Value'] = (close_price * lot_size)
      df.at[i, 'MDD_dollar']   = (close_price * lot_size) - initial_equity_value
      df.at[i, 'Holding period'] = hold_counter
      
      pnl = round(((close_price * lot_size) - sell_commission) - initial_equity_value,2)
      df.at[i, 'PNL'] = pnl.round(2)
      if pnl > 0:
        win_counter += 1
      pnl_list.append(pnl)
      bb_macd_tradelist.append([now_date, df_name, close_price, pos_opened, lot_size, (close_price*lot_size), pnl, hold_counter])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      lot_size = 0
      hold_counter = 0

    else:
      #df.loc[i, 'Position']       = 1
      df.loc[i, 'Equity Value']   = (now_open * lot_size)
      df.loc[i, 'Lot Size']       = lot_size
      df.loc[i, 'MDD_dollar']     = (now_open * lot_size) - initial_equity_value
      hold_counter = hold_counter + 1


  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].mean().round(2)
  avg_hold_period = df['Holding period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate

In [68]:
backtest_bbmacd(LABU)

('LABU', 3, 333.88, 111.29, 3.32, 65.67, 100.0)

In [69]:
bb_macd_tradelist

[[Timestamp('2022-02-24 00:00:00'), 'LABU', 12.63, True, 39.0, 492.18, 0, 0],
 [Timestamp('2022-02-25 00:00:00'),
  'LABU',
  16.13,
  False,
  39.0,
  629.0699999999999,
  135.32,
  37],
 [Timestamp('2022-07-13 00:00:00'), 'LABU', 8.21, True, 60.0, 492.01, 0, 0],
 [Timestamp('2022-07-22 00:00:00'),
  'LABU',
  9.61,
  False,
  60.0,
  576.5999999999999,
  82.18,
  100],
 [Timestamp('2022-10-13 00:00:00'), 'LABU', 6.195, True, 80.0, 494.81, 0, 0],
 [Timestamp('2022-10-18 00:00:00'),
  'LABU',
  7.68,
  False,
  80.0,
  614.4,
  116.38,
  60]]

In [70]:
LABU[LABU['Position']>0]

,Date,Open,High,Low,Close,Volume,200EMA,ATR,RSI_14,MACD,...,BBand_M,BBand_L,Position,Lot Size,Equity Value,MDD_dollar,PNL,Holding Period,Win Count,Holding period
36,2022-02-24,12.630,16.17,12.55,16.10,25034100,48.1549,2.6734,39.7897,-2.4529,...,16.038,14.0080,1,39.0,492.18,NaN,NaN,NaN,NaN,NaN
37,2022-02-25,16.130,16.50,14.97,16.50,19775900,47.8399,2.5917,40.9865,-2.3480,...,15.886,14.1526,2,39.0,629.07,136.89,135.32,NaN,NaN,37.0
131,2022-07-13,8.210,9.54,8.11,9.24,55591200,24.3893,1.0566,61.5945,0.7028,...,9.142,8.2535,1,60.0,492.01,NaN,NaN,NaN,NaN,NaN
138,2022-07-22,9.610,9.70,8.24,8.28,46212900,23.3406,1.0848,51.1877,0.5836,...,8.910,7.7071,2,60.0,576.60,84.59,82.18,NaN,NaN,100.0
196,2022-10-13,6.195,7.17,6.12,7.07,77095100,16.9153,0.8780,45.0568,-0.4908,...,6.754,6.3519,1,80.0,494.81,NaN,NaN,NaN,NaN,NaN
199,2022-10-18,7.680,7.80,7.10,7.30,70110300,16.6222,0.8760,48.1475,-0.3995,...,6.960,6.3629,2,80.0,614.40,119.59,116.38,NaN,NaN,60.0


In [71]:
bbmacd_result_list = []

for df in df_list:
  try:
    df_name = get_df_name(df)
    test_result = backtest_bbmacd(df)
    bbmacd_result_list.append(test_result)

    bbmacd_result_df = pd.DataFrame(bbmacd_result_list)
    bbmacd_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  except: 
    test_result_null = bbmacd_result_list.append([df_name, 0, 0, 0, 0, 0, 0])
    bbmacd_result_df = pd.DataFrame(bbmacd_result_list)
    bbmacd_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']

In [72]:
# PNL for Bollinger Bands + MACD Strategy

measure_date = df_list[1].loc[1, 'Date']
end_date = df_list[1].loc[df.index[-1], 'Date'] 
print('Trading Strategy: Bollinger Bands + MACD')
print('Trade period:', (measure_date.strftime('%Y-%m-%d')), 'to', end_date.strftime('%Y-%m-%d'))
ROI = ((bbmacd_result_df['Total PNL'].sum()/ initial_capital)*100).round(2)
MDD = bbmacd_result_df['Maximum Drawdown'].mean().round(2)
print('Total No of Trades Executed:', (bbmacd_result_df['No of Trade'].sum()))
print('Total PNL: USD', bbmacd_result_df['Total PNL'].sum().round(2))
print('Required Capital: USD', initial_capital)
print('ROI: ', ROI, '%')
print('Avg Win Rate: ', round(bbmacd_result_df[bbmacd_result_df['No of Trade'] > 0]['Win Rate %'].mean(),2), '%')
bbmacd_result_df[bbmacd_result_df['No of Trade'] > 0].sort_values(by=['Total PNL', 'Avg PNL/ Trade'], ascending=False)


Trading Strategy: Bollinger Bands + MACD
Trade period: 2022-01-04 to 2022-12-23
Total No of Trades Executed: 46
Total PNL: USD 3260.81
Required Capital: USD 7000
ROI:  46.58 %
Avg Win Rate:  82.26 %


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade,Maximum Drawdown,Avg Holding Days,Win Rate %
6,SOXL,4,468.80,117.20,3.79,56.75,100.00
7,SOXS,4,369.36,92.34,3.18,59.25,100.00
11,LABU,3,333.88,111.29,3.32,65.67,100.00
0,TQQQ,4,326.97,81.74,5.55,57.75,75.00
5,TNA,4,324.46,81.11,4.33,56.00,100.00
2,SPXL,3,236.61,78.87,5.78,77.00,100.00
3,SPXS,3,232.48,77.49,2.65,80.00,100.00
4,TZA,2,213.86,106.93,1.56,120.00,100.00
9,FAZ,3,174.73,58.24,1.50,79.67,66.67
10,LABD,3,151.58,50.53,0.83,79.00,100.00


In [73]:
LABU.to_csv('LABU.csv')
SOXL.to_csv('SOXL.csv')

In [74]:
def backtest_bbadxrsi(df):
  df_name = get_df_name(df)
  pos_opened = False
  open_price  = 0
  close_price = 0
  hold_counter = 0
  
  pnl = 0
  pnl_list = []
  
  lot_size = 0
  initial_equity_value = 0
  initial_buy_price = 0
  win_counter = 0
  stop_loss = initial_equity_value * 0.9
  profit_target = initial_equity_value * 1.2

  for i in range(len(df)):
    now_date        = df.loc[i, 'Date']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_200EMA      = df.loc[i, '200EMA']
    now_rsi_14      = df.loc[i, 'RSI_14']
    now_ADX         = df.loc[i, 'ADX']    
    now_MFI         = df.loc[i, 'MFI']
    now_willr       = df.loc[i, 'WILL%R']
    now_MACD        = df.loc[i, 'MACD']
    now_MACD_signal = df.loc[i, 'MACD_signal']
    now_bband_u     = df.loc[i, 'BBand_U']
    now_bband_m     = df.loc[i, 'BBand_M']
    now_bband_l     = df.loc[i, 'BBand_L']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']

    # opening a position
    if (pos_opened == False) and ((now_open < now_bband_l) and ((now_MFI > 50) and (now_rsi_6 < 35))):
      pos_opened = True
      open_price = now_open
      lot_size = pos_size//open_price
      buy_commission = (0.0049 + 0.005) * lot_size
      initial_equity_value = round(((lot_size * open_price) - buy_commission),2)
      initial_buy_price = initial_equity_value/ lot_size
      hold_counter = hold_counter + 1
      df.loc[i, 'Position'] = 1
      df.loc[i, 'Lot Size'] = lot_size
      df.loc[i, 'Equity Value'] = initial_equity_value
      bbadxrsi_trade_list.append([now_date, df_name, open_price, pos_opened, lot_size, initial_equity_value])
    
    # closing a position - by MACD signal, or last day of trading or hit stop loss
    elif ((pos_opened == True) and ((now_open > now_bband_u) or (now_MFI < 20) or (now_rsi_6 > 65)) or \
      (pos_opened == True) and (now_date == df.loc[(len(df)-1), 'Date'])):
      pos_opened = False
      close_price = now_open
      sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      pnl = round(((close_price * lot_size) - sell_commission) - initial_equity_value,2)
      df.loc[i, 'Position'] = 2
      df.loc[i, 'Lot Size'] = lot_size
      df.loc[i, 'Equity Value'] = (now_open * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - initial_equity_value
      df.loc[i, 'PNL'] = pnl.round(2)
      df.loc[i, 'Holding period'] = hold_counter
      if pnl > 0:
        win_counter += 1
      pnl_list.append(pnl)
      bbadxrsi_trade_list.append([now_date, df_name, close_price, pos_opened, lot_size, (close_price*lot_size)])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      initial_buy_price = 0
      lot_size = 0
      stop_loss = 0
      hold_counter = 0

    # calculating daily drawdowns
    #else (pos_opened == True):
    elif (pos_opened==True):
      df.loc[i, 'Equity Value']   = (now_close * lot_size)
      df.loc[i, 'MDD_dollar']     = (now_close * lot_size) - round(initial_equity_value,2)
      hold_counter = hold_counter + 1
 
  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].max().round(2)
  avg_hold_period = df['Holding period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate

In [75]:
bbadxrsi_result_list = []

for df in df_list:
  try:
    df_name = get_df_name(df)
    test_result = backtest_bbadxrsi(df)
    bbadxrsi_result_list.append(test_result)

    bbadxrsi_result_df = pd.DataFrame(bbadxrsi_result_list)
    bbadxrsi_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  except: 
    test_result_null = bbadxrsi_result_list.append([df_name, 0, 0, 0, 0, 0, 0])
    bbadxrsi_result_df = pd.DataFrame(bbadxrsi_result_list)
    bbadxrsi_result_df.columns = ['Ticker', 'No of Trade', 'Total PNL', 'Avg PNL/ Trade', 'Maximum Drawdown', 'Avg Holding Days', 'Win Rate %']
  

In [76]:
# PNL for Bollinger Bands + ADX + RSI Strategy

measure_date = df_list[1].loc[1, 'Date']
end_date = df_list[1].loc[df.index[-1], 'Date'] 
print('Trading Strategy: Bollinger Bands + ADX + RSI')
print('Trade period:', (measure_date.strftime('%Y-%m-%d')), 'to', end_date.strftime('%Y-%m-%d'))
ROI = ((bbadxrsi_result_df['Total PNL'].sum()/ initial_capital)*100).round(2)
MDD = bbadxrsi_result_df['Maximum Drawdown'].mean().round(2)
print('Total No of Trades Executed:', (bbadxrsi_result_df['No of Trade'].sum()))
print('Total PNL: USD', bbadxrsi_result_df['Total PNL'].sum().round(2))
print('Required Capital: USD', initial_capital)
print('ROI: ', ROI, '%')
print('Avg Win Rate: ', round(bbadxrsi_result_df[bbadxrsi_result_df['No of Trade'] > 0]['Win Rate %'].mean(),2), '%')
bbadxrsi_result_df[bbadxrsi_result_df['No of Trade'] > 0].sort_values(by=['Total PNL', 'Avg PNL/ Trade'], ascending=False)

Trading Strategy: Bollinger Bands + ADX + RSI
Trade period: 2022-01-04 to 2022-12-23
Total No of Trades Executed: 0
Total PNL: USD 0
Required Capital: USD 7000
ROI:  0.0 %
Avg Win Rate:  nan %


,Ticker,No of Trade,Total PNL,Avg PNL/ Trade,Maximum Drawdown,Avg Holding Days,Win Rate %


In [77]:
df_list = [TQQQ, SQQQ, SPXL, SPXS, TZA, TNA, SOXL, SOXS, FAS, FAZ, LABD, LABU, EDZ, EDC]

In [78]:
def backtest_bbmacd(df):
  df_name = get_df_name(df)
  pos_opened = False
  open_price  = 0
  close_price = 0
  hold_counter = 0
  
  pnl = 0
  pnl_list = []
  
  lot_size = 0
  initial_equity_value = 0
  initial_buy_price = 0
  win_counter = 0
  stop_loss = initial_equity_value * 0.9
  profit_target = initial_equity_value * 1.2

  for i in range(len(df)):
    now_date        = df.loc[i, 'Date']
    now_open        = df.loc[i, 'Open']
    now_close       = df.loc[i, 'Close']
    now_200EMA      = df.loc[i, '200EMA']
    now_rsi_14      = df.loc[i, 'RSI_14']
    now_MACD        = df.loc[i, 'MACD']
    now_MACD_signal = df.loc[i, 'MACD_signal']
    now_MACD_hist   = df.loc[i, 'MACD_hist']
    now_bband_u     = df.loc[i, 'BBand_U']
    now_bband_m     = df.loc[i, 'BBand_M']
    now_bband_l     = df.loc[i, 'BBand_L']
    now_position    = df.loc[i, 'Position']
    now_equity_val  = df.loc[i, 'Equity Value']
    now_mdd_dollar  = df.loc[i, 'MDD_dollar']

    # open trade conditions
    open_trade_cond = ((now_open < now_bband_l) and (now_MACD > now_MACD_signal) and (now_MACD_hist > 0))

    # close trade conditions
    close_trade_cond = (now_MACD < now_MACD_signal)
    profit_cond = (now_open * lot_size) > profit_target
    stop_loss_cond = (now_open * lot_size) < stop_loss
    force_close_cond = (now_date == df.loc[(len(df)-1), 'Date'])  

    # opening a position
    if (pos_opened == False and open_trade_cond):
      pos_opened = True
      open_price = now_open
      lot_size = pos_size//open_price
      buy_commission = (0.0049 + 0.005) * lot_size
      initial_equity_value = round(((lot_size * open_price) - buy_commission),2)
      initial_buy_price = initial_equity_value/ lot_size
      hold_counter = hold_counter + 1
      df.at[i, 'Position'] = 1
      df.at[i, 'Lot Size'] = lot_size
      df.at[i, 'Equity Value'] = initial_equity_value
      bb_macd_tradelist.append([now_date, df_name, open_price, pos_opened, lot_size, initial_equity_value])

    # calculating daily drawdowns
    if pos_opened == True and not (close_trade_cond or profit_cond or stop_loss_cond or force_close_cond):
      df.at[i, 'Equity Value']   = (now_open * lot_size)
      df.at[i, 'Lot Size']       = lot_size
      df.at[i, 'MDD_dollar']     = (now_open * lot_size) - initial_equity_value
      hold_counter = hold_counter + 1

    # closing a position - by MACD signal, or last day of trading or hit stop loss
    if pos_opened == True and (close_trade_cond or profit_cond or stop_loss_cond or force_close_cond):
      pos_opened = False
      close_price = now_open
      sell_commission = (0.04 * lot_size) + (close_price * lot_size * 0.0000229)
      pnl = round(((close_price * lot_size) - sell_commission) - initial_equity_value,2)
      if pnl > 0:
        win_counter += 1
      df.at[i, 'Position'] = 2
      df.at[i, 'Lot Size'] = lot_size
      df.at[i, 'Equity Value'] = (now_open * lot_size)
      df.at[i, 'MDD_dollar']     = (close_price * lot_size) - initial_equity_value
      df.at[i, 'PNL'] = pnl.round(2)
      df.at[i, 'Holding period'] = hold_counter
      pnl_list.append(pnl)
      bb_macd_tradelist.append([now_date, df_name, close_price, pos_opened, lot_size, (close_price*lot_size), pnl, hold_counter])

      # reset values
      open_price = 0
      close_price = 0
      initial_equity_value = 0
      initial_buy_price = 0
      lot_size = 0
      stop_loss = 0
      profit_target = 0
      hold_counter = 0

    #else:
    #  df.at[i, 'Equity Value']   = (now_open * lot_size)
    #  df.at[i, 'Lot Size']       = lot_size
    #  df.at[i, 'MDD_dollar']     = (now_open * lot_size) - initial_equity_value
    #  hold_counter = hold_counter + 1


  total_profit = round(sum(pnl_list), 2)
  num_of_trade = round(len(pnl_list), 2)
  avg_pnl = round(total_profit/ num_of_trade, 2) if num_of_trade else 0
  max_mdd = df['MDD_dollar'].mean().round(2)
  avg_hold_period = df['Holding period'].mean().round(2)
  win_rate = round(win_counter/ num_of_trade * 100,2)

  return df_name, num_of_trade, total_profit, avg_pnl, max_mdd, avg_hold_period, win_rate